In [1]:
using POMDPs
using Random 
using Parameters
using StaticArrays
using Distributions
using FIB
using MCTS

In [2]:
const Vec2 = SVector{2, Float64}
const Vec3 = SVector{3, Float64}

SArray{Tuple{3},Float64,1,3}

In [3]:
struct state
    uavPose::Vec3
    uavHeading::Float64 # radius
    targetPose::Vec2
end

struct action
    xy_speed::Float64 # m/s
    z_speed::Float64
    angle::Float64 # radius
end

mutable struct UAVMDP <: MDP{state, action}

    target_velocity::Vec2
    target_std::Float64

    r_outScene::Float64
    r_action::Float64
    r_reach::Float64

    discount::Float64
    
    boundary::Int
    landing_r::Int

    mission_end::Bool
end

In [4]:
function POMDPs.generate_s(pp::UAVMDP, s::state, a::action, rng::AbstractRNG)
    p = UAVMDP(pp)
    # calculate target state
    target_dt_distance = p.dt*p.target_velocity
    sensor_noise = rand(Normal(0, p.target_std), 2)
    curr_targ = s.targetPose + target_dt_distance + sensor_noise # next_target_pos(p, s.targetPose)
    # calculate UAV state
    curr_angle = s.uavHeading + a.angle
    xy_dt_distance = p.dt*a.xy_speed*SVector(cos(curr_angle), sin(curr_angle)) # careful
    z_dt_distance = p.dt*a.z_speed
    xyz_dt_distance = SVector(xy_dt_distance[1], xy_dt_distance[2], z_dt_distance)
    curr_pos = s.uavPose + xyz_dt_distance
    return states(curr_pos, curr_angle, curr_targ)
end

In [5]:
function POMDPs.reward(pp::UAVMDP, s::state, a::action, rng::AbstractRNG)
    p = UAVMDP(pp)
    
    distance_to_target = sqrt((s.uavPose[1]-s.targetPose[1])^2 + (s.uavPose[2]-s.targetPose[2])^2 + s.uavPose[3]^2)
    
    reward = p.r_action + 1/distance_to_target
#     out scene punish 
    if s.uavPose[1]>p.boundary || s.uavPose[2]>p.boundary
        reward = reward + p.r_outScene
        p.mission_end = true
    end
    
    if distance_to_target < p.landing_r
        reward = reward + p.r_reach
        p.mission_end = true
    end
    return reward
end

In [6]:
POMDPs.initialstate_distribution(p::UAVMDP) = Deterministic(false)
solver = MCTSSolver(n_iterations=50, depth=20, exploration_constant=5.0)
UAV = UAVMDP(SVector(0.5,0.5),0.1,-70,-10,100,0.95,100,1,false)

UAVMDP([0.5, 0.5], 0.1, -70.0, -10.0, 100.0, 0.95, 100, 1, false)

In [7]:
policy = solve(solver, UAV)

MCTSPlanner{UAVMDP,state,action,MCTS.SolvedRolloutEstimator{POMDPPolicies.RandomPolicy{MersenneTwister,UAVMDP,BeliefUpdaters.NothingUpdater},MersenneTwister},MersenneTwister}(MCTSSolver(50, 20, 5.0, MersenneTwister(UInt32[0xe40cf386, 0x97b7e968, 0x78d7dd21, 0x05c54d15], Random.DSFMT.DSFMT_state(Int32[-98281423, 1073502871, -1994156596, 1073720132, 193631232, 1073530068, 662624214, 1072948239, -324277788, 1073311180  …  299767352, 1073122290, -397553185, 1073485599, -1297620421, -1722936499, -1857268063, -906139162, 382, 0]), [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], UInt128[0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x000000000000000000000000000000